In [1]:
# 调用相关库
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import pandas as pd
from math import sqrt
import random
from numpy import concatenate
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score
import scipy as spy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas import DataFrame
from pandas import concat
import numpy
from scipy.io import savemat,loadmat
import os
from sklearn.svm import SVR  
from scipy.spatial.distance import cdist
from numpy import dot, exp

In [2]:
class DELM(object):

    def __init__(self, input_size,hidden_sizes,output_size,C,TF):
        # 初始化
        '''
            input_size:输入层节点数
            hidden_sizes:各隐含层节点数，也是各elm-ae的隐含层节点数
            output_size:输出层节点数
            C ： L2 正则化系数
            TF： 激活函数
        '''
        self._sizes = hidden_sizes
        self._l2penalty=C
        self.TF=TF 
        self.elmae=[]
        self.w=[]
        
        for i,size in enumerate(self._sizes + [output_size]):
            
            # 采样正交随机数进行网络权重初始化
            
            if input_size>=size:
                IW=spy.linalg.orth(np.random.rand(input_size,size)*2-1)
            else:
                IW=(np.random.rand(input_size,size)*2-1).T
                IW=spy.linalg.orth(IW).T
            # bias
            B=spy.linalg.orth(np.random.rand(size,1)*2-1)
            
            self.w.append(IW)
            
            if i< len(self._sizes + [output_size])-1:
                weight={'w':IW,'b':B}
                self.elmae.append(weight)
            
            input_size = size
    #无监督预训练各个ELM-AE
    def pretrain(self,X):
        self.elm_ae_weigth_list=[]#保存训练好的elmae的输出层权重 ，用于DELM初始化
        nsample=X.shape[0]
        for i in range(len(self.elmae)):
            IW=self.elmae[i]['w']
            B=self.elmae[i]['b']
            
            bias_matrix=np.tile(B,nsample).T
            
            tempH = np.dot(X,IW)+bias_matrix
            
            if self.TF=='sigmoid':
                H = 1. / (1. + np.exp(-tempH))
                
            elif self.TF=='sin':
                H=np.sin(tempH)
            # cal the output weights by least square method
            interval=(np.dot(H.T,H)+np.eye(IW.shape[1])/self._l2penalty)
            beta = np.dot(np.dot(np.linalg.pinv(interval) ,H.T),X)
            
            
            W=np.linalg.pinv(beta)
            # W = beta.T
            self.elm_ae_weigth_list.append(W)
            X=X.dot(W)
#                
    # 将训练好的elm-ae的输出层权重用于初始化delm
    def load_from_elm_aes(self):
        for i in range(len(self._sizes)):
            self.w[i] = self.elm_ae_weigth_list[i]
        
    # Training delm
    def train(self,X,Y):
        self.load_from_elm_aes()#注释掉这句 就不会采用训练好的elm-ae输出层权重对delm进行初始化
        
        
        H=[]
        input_=X
        a=len(self._sizes)
        
        for i in range(a):
            H=np.dot(input_,self.w[i])
            input_=H

        interval=np.dot(H.T,H)+np.eye(H.shape[1])/self._l2penalty
        
        beta = np.dot(np.dot(np.linalg.pinv(interval) ,H.T),Y)
        
        self.w[-1]=beta
        
    def predict(self,teX):
        H=[]
        input_=teX
        a=len(self.w)
        
        for i in range(a):
            H=np.dot(input_,self.w[i])
            input_=H
        return H

In [3]:
# 转换成监督数据，将每个特征转换成21列数据，20->1，20组预测一组
def series_to_supervised(data, n_in=20, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]  #计算特征的数量
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    # 将20组输入数据依次向下移动20,19,...3,2,1行，将数据加入cols列表（技巧：(n_in, 0, -1)中的-1指倒序循环，步长为1）
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    # 将一组输出数据加入cols列表（技巧：其中i=0）
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # cols列表(list)中现在有21块经过下移后的数据(即：df(-20),df(-19),...df(-2),df(-1),df)，将四块数据按列 并排合并
    agg = pd.concat(cols, axis=1)
    # 给合并后的数据添加列名
    agg.columns = names
    # print(agg)
    # 删除NaN值列
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [4]:
values=pd.read_csv('train.csv').values[:5000,-1]
values=values.reshape(-1,1)
n_features=1 #特征数量

In [5]:
# 标准化
scaler =StandardScaler()
scaled = scaler.fit_transform(values)


In [6]:
n_in = 3
n_out = 1
# 构造一个20->1的监督学习型数据
reframed = series_to_supervised(scaled, n_in=n_in, n_out=n_out)
values=reframed.values

In [7]:
#训练集和测试集的划分比例
ratio=0.8 
N=int(len(values) *ratio)
train = values[:N, :]
test = values[N:, :]

# 划分输入和输出,split into input and outputs
n_obs = n_in * n_features
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(3997, 3) (3997,) (1000, 3) (1000,)


In [8]:
train_y=train_y.reshape(-1,1)
train_y.shape

(3997, 1)

In [33]:
# In[] 参数设置
hidden_sizes = [8,5]  #几个数就是几个隐含层，n个隐含层各自的节点就是[n1;n2;n3;n4;....nn]
input_size = train_X.shape[1] #输入层节点数
output_size= train_y.shape[1]#输出层节点数
C=np.inf #L2正则化系数
TF='sigmoid'#激活函数       "sig"

In [34]:
# In[3]: 训练
Net = DELM(input_size,hidden_sizes, output_size, C,TF) #construct a delm
Net.pretrain(train_X)#train elmae
Net.train(train_X,train_y)#train delm

In [35]:
#预测
yhat = Net.predict(test_X)
yhat=yhat.reshape(-1,1)
# 对预测结果进行反归一化
inv_yhat = scaler.inverse_transform(yhat)
# 将真实列据进行反归一化
test_y=test_y.reshape(-1,1)
inv_y = scaler.inverse_transform(test_y)

In [36]:
# 计算RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.7f' % rmse)
print('Test MAE: %.7f' % mean_absolute_error(inv_y, inv_yhat))
print('Test R2: %.7f' % r2_score(inv_y, inv_yhat))

Test RMSE: 15.5769520
Test MAE: 12.4538736
Test R2: 0.0148082


In [ ]:
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
plt.rcParams['font.sans-serif']=['Simhei']  #解决画图中文不显示问题
plt.figure(figsize=(6,4),dpi=600)
x=range(1, len(inv_y) + 1)
plt.xticks(x[::50])
plt.tick_params(labelsize=10)  #改变刻度字体大小
plt.plot(x, inv_y, linestyle="--",color='g',linewidth=1, label='Acture')
plt.plot(x, inv_yhat,  color='red',linestyle="--",linewidth=1,label='DELM')
plt.rcParams.update({'font.size': 10})  #改变图例里面的字体大小
plt.legend(loc='upper right')
plt.xlabel("样本点",fontsize=10)
plt.ylabel("值",fontsize=10)
# plt.xlim(xmin=600,xmax=700)  #显示600-1000的值   局部放大有利于观察
# plt.savefig('台湾VMD-LSTM震级预测结果图.png')
plt.show()

In [ ]:
#保存预测结果
savemat('delm_result.mat',{'true':inv_y,'pred':inv_yhat})